---
title: "Decision by sampling: Monte Carlo approaches to dealing with uncertainty"
summary: "This is how I'm dealing with my anxiety"
date: 2020-03-27
source: jupyter
---


Dealing with uncertainty is hard.
Algebra is also hard.
This post is a gentle introduction to a technique
that lets you do the former,
without having to do too much of the latter.
To do this, we'll use two useful concepts from computer science and statistics:
**Monte Carlo sampling**, and **Bayesian probability distributions**.

> Note:
> I wrote the first draft of this post early in the COVID lockdown,
> finished it 6 weeks in. There will be coronavirus-related examples,
> and the writing may be a little unhinged.
> Be warned.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib import rcParams
import seaborn as sns
sns.set_style('whitegrid')
rcParams['figure.figsize'] = (6, 4)
rcParams['font.size'] = 18
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

from ipywidgets import interact, FloatSlider
def fs(value, min, max, step, description=None):
    '''Shorthand for ipywidgets.FloatSlider'''
    return FloatSlider(value=value, min=min, max=max, step=step, 
                       description=description, continuous_update=False)

# Monte Carlo Sampling

In *Monte Carlo* approaches, we use random simulations to answer questions 
that might otherwise require some difficult equations.
Confusingly, they're also known in some fields 
as *numerical* approaches, and are contrasted with *analytic* approaches,
where you just work out the correct equation.
[Wikipedia tells us](https://en.wikipedia.org/wiki/Monte_Carlo_method#History) that,
yes, Monte Carlo methods are named after the casino.
I won't be talking in this post about 
[Markov Chain Monte Carlo](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo),
a particular Monte Carlo method that comes up a lot in Bayesian statistics.

Here's a simple Monte Carlo example.
Let's say you want to know the area of a circle
with a radius of $r$.
We'll use a unit circle, $r=1$, in this example.

In [ ]:
def circle_plot():
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.hlines(1, -1, 1)
    plt.vlines(1, -1, 1)
    plt.scatter(0, 0, marker='+', color='k')
    plt.xlim(-1, 1)
    plt.ylim(-1, 1)
    circle = plt.Circle((0, 0), 1, facecolor='None', edgecolor='r')
    ax.add_artist(circle)
    return fig, ax

circle_plot();

Analytically, you know that the answer is $\pi r^2$. 
What if we didn't know this equation?
The Monte Carlo solution is as follows.
We know that the area of the bounding square is $2r \times 2r = 4r^2$
We need to figure out what proportion of this square is taken up by the circle.
To find out, we randomly select a large number of points in the square,
and check if they're within $r$ of the center point $[0, 0]$.

In [ ]:
n = 1000 # Number of points to simulate
x = np.random.uniform(low=-1, high=1, size=n)
y = np.random.uniform(low=-1, high=1, size=n)
# Distance from center (Pythagoras)
dist_from_origin = np.sqrt(x**2 + y**2)
# Check is distance is less than radius
is_in_circle = dist_from_origin < 1

# Plot results
circle_plot()
plt.scatter(x[is_in_circle], y[is_in_circle], color='b', s=2)     # Points in circle
plt.scatter(x[~is_in_circle], y[~is_in_circle], color='k', s=2);  # Points outside circle

m = is_in_circle.mean()
print('%.4f of points are in the circle' % m)

Since the area of the square is $4r^2$,
and the circle takes up ~$0.78$ of the square,
the area of the circle is roughly $0.78 \times 4r^2 = 3.14r^2$.
We've discovered $\pi$.

## Bayesian Probability Distributions

The term **Bayesian statistics** refers to a whole family of approach to statistical inference.
What is common to all of these approaches is that they take probabilities 
to be statements about *subjective beliefs*.
This means a Bayesian doctor can say things like 
"*I'm 90% sure this patient has COVID-19*",
while a non-Bayesian doctor could only say something like 
"*for every 100 patients with these symptoms, 90 will have it*".
If this differences doesn't make much sense to you,
fear not, because a) you're not alone, and 
b) it doesn't matter for this post.

The Bayesian approach gives us a useful way of thinking about uncertainty.
If we're unsure about some number,
we can replace it with a 
[*probability mass function*](https://en.wikipedia.org/wiki/Probability_mass_function)
(if the number is discrete, like a count)
or a 
[*probability density function*](https://en.wikipedia.org/wiki/Probability_density_function) 
(if the number is continuous, like a measurement)
over possible values.

For example, let's say you don't know how tall I am.
I'm an adult Irish male,
and we know that the heights of Irish men in general are 
normally distributed,
with a mean of 70 and a standard deviation of 4 inches.
This means you can use the distribution 
$Normal(70, 4)$,
where $Normal(\mu, \sigma)$ standards for the 
Normal probability distribution function with mean $\mu$ and standard deviation $\sigma$.

In [ ]:
heights_to_plot = np.arange(50, 90, .1)
pdf = stats.norm.pdf(heights_to_plot, 70, 4)
plt.plot(heights_to_plot, pdf)
plt.fill_between(heights_to_plot, pdf, 0, alpha=.1)
plt.ylim(0, .11)
plt.xlabel('Possible Heights (inches)')
plt.ylabel('Belief that Eoin\nmight be this tall')

$Normal(\mu, \sigma)$
is shorthand for a relatively complicated function

$$
Normal(\mu, \sigma) = \frac{1}{\sigma\sqrt{2\pi}} 
e^{\frac{1}{2}(\frac{x - \mu}{\sigma})^2}
$$

where $x$ is each particular height.
I said at the outset that we wouldn't do very much maths.
Luckily, you don't have to actual use this function.
Instead, we can use our computers to quickly and easily
**sample** a large number of values from this distribution,
and use these samples as an estimate of my height.

In [ ]:
N = 10000
height_samples = np.random.normal(70, 4, N)
height_samples

In [ ]:
plt.hist(height_samples, bins=np.arange(50, 90, 1))
plt.xlabel('Height (Inches)')
plt.ylabel('Samples where\nEoin is this tall');

Now that we have these samples, we can use them to answer questions.
For instance, what's the probability that I'm taller than 6 foot (72 inches)?
To find out, you need to find how much of the distribution is above this value.
This is easy with the samples: it's the proportion of samples greater than this value.

In [ ]:
print('P(Eoin > 72 inches) = %.2f' % np.mean(height_samples > 72))

One more:
what's the probability that I'm taller than 5 foot 6, but less than 6 foot?

In [ ]:
p = np.mean((height_samples > 66) & (height_samples < 72))
print('P(66 < Eoin < 72) = %.2f' % p)

So far, this isn't very exciting.
You could obtain the same answers by just checking 
what proportion of Irish men are more than 6 foot tall, 
without using impressive terms like "Bayesian".

This approaches comes into its own is
when we must combine multiple sources of information.
In Bayesian statistical modelling,
this usually means combining our prior beliefs 
(like your beliefs about how tall I'm likely to be)
with multiple data points.
Here, we're going to look at a simpler example:
making predictions based on uncertain knowledge.
To do it, we're going to have to start talking about COVID-19. Sorry.


## Incubation Periods

A few weeks ago, when I wrote the first draft of this post,
the whole world was talking about incubation periods,
specifically that it took up to two weeks for COVID symptoms to develop 
after contact with a carrier.
This prompted some confusing infographics, like the one below from the BBC.

<img style="max-width: 400px" src="https://ichef.bbci.co.uk/news/624/cpsprodpb/58B3/production/_111370722_self_isolation_timetab_640_3x-nc.png">


Where does the 14 days figure come from?
[Lauer et al (2020)](https://annals.org/aim/fullarticle/2762808/incubation-period-coronavirus-disease-2019-covid-19-from-publicly-reported) 
analysed incubation times from 181 cases, 
and concluded that the times followed a Log-Normal distribution
with $\mu = 1.621$ and $\sigma = 0.418$.
Note that these parameters are the mean and standard deviation
of the log of the incubation time distribution,
rather than the mean and standard deviation of the 
incubation times themselves.

$$
\alpha \sim \text{Log-Normal}(1.621, 0.418)
$$


<img style="max-width: 400px" src="https://raw.githubusercontent.com/HopkinsIDD/ncov_incubation/master/README_files/figure-markdown_strict/dic-plots-1.png">

They also posted their data and code to [this GitHub repository](https://github.com/HopkinsIDD/ncov_incubation).

As we've seen above, since we don't know in advance
the exact incubation time in individual cases,
we can use simulated samples from this distribution instead.

In [ ]:
def histogram(x, label=None, bins=None, density=True):
    '''We'll be doing lots of histograms, so here's a
    funciton that makes them easier'''
    plt.hist(x, bins=bins, density=density);
    if label:
        plt.xlabel(label)
    if not density:
        plt.yticks([])
        plt.gca().axes.spines['left'].set_visible(False)
    else:
        plt.ylabel('Probability')
        
        
n = 10000
incubation_mu = 1.621
incubation_sigma = 0.418
incubation_times = np.random.lognormal(incubation_mu, incubation_sigma, n)
## We could also use
# incubation_times = stats.lognorm(loc=0, scale=np.exp(incubation_mu), s=incubation_sigma).rvs(n)
histogram(incubation_times, u'Incubation time in days', bins=range(20))

Note that we've set `density=True`, so instead of showing the number of simulated values in each bin,
the histogram shows the *proportion* of vales in each bin.

##  Have you caught it?

We can already use these samples to answer some questions.

Let's say you're in full quarantine.
Just before going in,
you had to be somewhere risky,
where there was a probability $\alpha$ 
you picked up the virus.
We'll start by assuming there was a fifty-fifty
of picking up the virus, $\alpha = .5$.
Let's also assume for now (wrongly) that if you have the virus,
you're guaranteed to develop symptoms eventually.

After $d$ days in quarantine, you still haven't developed any symptoms.
What is the probability that you've picked up the virus?
We can work this out analytically,
and we do below,
but it's much easier to just run simulations here.
To do this, we run $N$ simulations of the scenario described above.
The rules of the simulations are as follows.

- In each simulation, the probability of catching the virus is  $\alpha$,
  and the probability of not catching it is $1 - \alpha$.
  This is known as a [*Bernoulli distribution*](https://en.wikipedia.org/wiki/Bernoulli_distribution), with probability $\alpha$.
- In simulations where you do catch it, the length of your incubation period $\tau$
  is sampled randomly from the distribution 
  $\text{Log-Normal}(1.621, 0.418)$
- If you do have the virus, and $d \gt \tau$,
  you have symptoms. Otherwise, you don't.

We want to estimate $P(\text{Has virus} | \text{No symptoms by day }d)$.
To do so, we check on how many of the simulations where no symptoms developed by day $d$
do you in fact have the virus.
Here's the code after 5 days, $d = 5$.

In [ ]:
# Set our parameters
N = 10000   # Number of simulations
alpha = .5  # P(picked up virus) 
d = 5       # Days in quarantine.

# Simulations where you're infected
# (1 if infected, 0 otherwise)
is_infected = np.random.binomial(1, alpha, n)

# Incubation times for people who are infected
incubation_times = np.random.lognormal(incubation_mu, incubation_sigma, n)
# Simulations where you have symptoms by day d.
symptoms_today = np.where(is_infected & (incubation_times < d), 1, 0)

# In how many of the simulations where you show no symptoms today
# do you turn out to actually be infected?
p_infected = is_infected[symptoms_today == 0].mean()
print('P(Infected) = %.2f' % p_infected)

Next, let's calculate it over a range of days
(😊 = No symptoms).

In [ ]:
# Ineffecient code
days = np.arange(0, 20)
probabilities = []
for d in days:
    symptoms_today = np.where(is_infected & (incubation_times < d), 1, 0)
    p_infected = is_infected[symptoms_today==0].mean()
    probabilities.append(p_infected)
plt.plot(days, probabilities)
plt.ylim(0, 1)
plt.xlim(0, 20)
plt.xlabel('Days in Quarantine')
plt.ylabel('P(Infected | 😊)')
plt.show()

Finally, just to show off, let's make our code more efficient, wrap it in a function,
and then wrap that in an interactive widget.

In [ ]:
def plot_p_infected(alpha: float, 
                   incubation_mu: float=1.6, incubation_sigma: float=0.4):
    '''Plot posterior probability that you're infected if you 
    remain symptom free over a number of days since contact.

    Arguments:
    - prior_p_infected: Prior probability of infection.
    - prob_symptoms_if_infected: Probability of symptoms if infected
    - incubation_mu: Log-mean parameter for incubution time distribution (default = 1.6)
    - incubation_sigma: Log-SD parameter for incubution time distribution (default = 0.4)
    
    Returns nothing, but plots a figure.
    '''
    n = 1000
    days = range(0, 20)
    
    is_infected = np.random.binomial(1, alpha, n)
    incubation_times = np.random.lognormal(incubation_mu, incubation_sigma, n)
    def get_p_for_day(d):
        '''Calculate P(Infected) after d days'''
        symptoms_today = np.where(is_infected & (incubation_times < d), 1, 0)
        return is_infected[symptoms_today==0].mean()
    probabilities = [get_p_for_day(d) for d in days]
    plt.plot(days, probabilities)
    plt.ylim(0, 1)
    plt.xlabel('Days since contact')
    plt.ylabel('P(Infected | 😊)')
    plt.show()

interact(plot_p_infected, 
         alpha = fs(.5, 0, 1, .1, 'α'),
         incubation_mu    = fs(1.6, 0, 5, .1, 'Incubation μ'),
         incubation_sigma = fs(0.4, 0, 1, .1, 'Incubation σ'));



## Room for Improvement

There are quite a few assumptions in this analysis that aren't right.
For example, it assumes:


- Everyone who has the virus will develop symptoms.
- Symptoms are obvious. 
  In reality, knowing whether or not you have the symptoms is a signal detection problem,
  and you need to include that uncertainty in your analysis.
- There is no other possible cause of COVID-like symptoms.


### Analytic Solution (Optional)

If you want to see the analytic solution to this problem, [check this endnote](#endnote1)



##  Am I still infectious?

Let's try a slightly more complicated example.

The infographic above makes a two strong assumptions.
First, it assumes that the incubation period is 14 days,
so that a person stops being infectious 14 days after the pick up the virus.
Second, it assumes that on Day 1, when Mum gets sick,
everyone else in the family picks up the virus from her immediately.
I don't know what this is called, so let's call it the *acquisition time*:
this infographic assumes an acquisition time of 0 days.
Third, it assumes that once you show symptoms, you're contagious for 7 days.
We'll call this the *recovery time*, although we won't worry about it in this post.

These are reasonable assumptions, since these are our best estimates for how these things work.
However, in reality we're uncertain about all of these things,
since these things will take longer in some cases than in others.
It also assumes that the recovery time starts counting down from the day you first show symptoms,
rather than the day you stop showing symptoms.
I don't know if that's a good assumption -
what if your symptoms last more than a week? - 
but I'm going to ignore it for now.

We can make the model used in this infographic explicit as follows.
Let's call the acquisition period $\tau_A$,
and incubation period $\tau_B$.
$d_1$ is the day on which the first person in the family (Mum) shows symptoms.
The graphic assumes $\tau_A = 0$, and $\tau_B = 14$.

If you do not develop symptoms,
the model states that you are contagious until day $\tau_C = d_1 + \tau_A + \tau_B$
(day of first infection plus time taken to acquire the virus yourself plus
time it would take for symptoms to develop if you have acquired it),
and not contagious afterwards.
Note we're not using the same notation as earlier in this post.

We've already seen that our best estimate of the incubation time is just

$$
\tau_B \sim \text{Log-Normal}(1.621, 0.418)
$$

In [ ]:
histogram(incubation_times, 'Incubation times (β)', bins=range(20))


I don't know of any empirical estimates of the acquisition time $\alpha$, which is assumed to be 0 days here.
It's useful to reframe this in terms of the *acquisition probability*, $\theta_A$:
if you wake up without the virus, but someone in your house has it,
what is the probability that you'll catch it that day? 
The acquisition time, $\tau_A$, follows a 
[geometric distribution](https://en.wikipedia.org/wiki/Geometric_distribution) 
(the first kind discussed on the Wikipedia page), 
with success probability $\theta_A$
("success" here meaning you successfully acquire the virus. I didn't choose this terminology).
The average acquisition time in this case is just $\frac{1}{\theta_A}$.

In [ ]:
thetas = [.25, .5, .75, .9999]
times = np.arange(0, 7, 1)
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

plt.sca(axes[0])
for theta in thetas:
    # Offset by one
    d = stats.geom(theta).pmf(times+1) 
    plt.plot(times, d, '-o', label='$\\theta_A = $%.2f' % theta)
plt.xlabel('Days since first infection')
plt.ylim(0, 1.1)
plt.legend()
plt.ylabel('P(Get infected today)')

plt.sca(axes[1])
for theta in thetas:
    d = stats.geom(theta).cdf(times+1)
    plt.plot(times, d, '-o', label='$\\theta_A = $%.2f' % theta)
plt.xlabel('Days since first infection')
plt.ylim(0, 1.1)
plt.legend()
plt.ylabel('P(Infected by end of today)')
plt.tight_layout()
plt.show()

For now, let's assume you have a 80% chance of picking up the virus
for every day you spend living with someone who is contagious:
$\theta_A = 0.8$.
We can then obtain a distribution over likely acquisition times, $\beta$
by sampling from a geometric distribution with probability parameter $0.8$.

$$
\tau_A \sim \text{Geometric}(0.8)
$$

In [ ]:
def sample_acq_times(acq_prob, n=n):
    return np.random.geometric(acq_prob, n) - 1 # Offset by one

acquisition_prob = .8
acquisition_times = sample_acq_times(acquisition_prob)
histogram(acquisition_times, 'Acquisition time in days ($\\tau_A$)', bins=range(0, 8))
plt.ylim(0, 1);

### Combining Distributions

We now have samples from distributions representing our uncertainty
about the acquisition time $\tau_A$, and the incubation period $\tau_B$.
We want to calculate the distribution of the isolation time, $\tau_C = d_1 + \tau_A + \tau_B$.
To do this, we just apply this formula to each individual sample:
$\tau_C^i = d_1 + \tau_A^i + \tau_B^i$, for $i$ in $1, 2, \dots, n$.

In [ ]:
isolation_times = acquisition_times + incubation_times
histogram(isolation_times, 'Isolation time in days (τ = α + β)', bins=range(30))

With this distribution, we can ask answer questions like

- What's the probability of still being contagious after 7 days? After 14 days?
- How long must you wait to have a 95% chance of not being contagious?

In [ ]:
def p_still_contagious(isolation_times, days):
    '''What is the probability you\'re contagious after `days` days?'''
    p = np.mean(isolation_times > days)
    return 'P(Contagious) after %i days\t= %.3f' % (days, p)
    
for days in [2, 7, 14]:
    print(p_still_contagious(isolation_times, days))

In [ ]:
def days_until_probability(isolation_times, prob):
    '''How many days must you wait to have this probability of being non-contagious?'''
    days = np.percentile(isolation_times, prob*100) # Percentile expects values 0-100
    return 'P(Not contagious) = %.3f after %i days' % (prob, days)

for prob in [.5, .75, .9, .99]:
    print(days_until_probability(isolation_times, prob))

Of course, the value of $\theta = 0.8$ was only a guess.
How much do our conclusions depend on these parameters?
To find out, we create a function that takes these parameter values as inputs,
and outputs a distribution over isolation times $\tau_C$,
using the code above.

In [ ]:
def infer_isolation_times(mu_incubation_time, sigma_incubation_time, acquisition_prob, n=10000):
    incubation_times = np.random.lognormal(mu_incubation_time, sigma_incubation_time, n)
    acquisition_times = sample_acq_times(acquisition_prob, n)
    isolation_times = acquisition_times + incubation_times
    return isolation_times

Better still, we can wrap this in another function that takes these parameters and produces a histogram and some summary values.

In [ ]:
def plot_isolation_times(isolation_times):
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    plt.sca(axes[0])
    plt.hist(isolation_times, bins=20)
    plt.xlabel('Isolation times (days)')
    plt.yticks([])
    plt.gca().axes.spines['left'].set_visible(False)
    plt.sca(axes[1])
    q = np.linspace(0, 1, 50)
    d = np.percentile(isolation_times, q*100)
    plt.ylim(0, 1.05)
    plt.plot(d, q)
    for ax in axes:
        ax.set_xlim(0, 20)
        ax.vlines(14, *ax.get_ylim(), linestyle='dashed')
    plt.xlabel('Time since first symptoms in house')
    plt.ylabel('P(No longer contagious)')
    
def show_isolation_times(mu_incubation_time=1.621, 
                         sigma_incubation_time=0.418, 
                         acquisition_prob=0.9):
    isolation_times = infer_isolation_times(mu_incubation_time, sigma_incubation_time, acquisition_prob, n=1000)
    plot_isolation_times(isolation_times)

show_isolation_times()        

...and wrap the whole thing in an interactive widget.

In [ ]:
interact(show_isolation_times, 
         mu_incubation_time   = fs(1.6, 0, 5, .1, 'Incubation μ'),
         sigma_incubation_time = fs(0.4, 0, 1, .1, 'Incubation σ'),
         acquisition_prob     = fs(.9, 0, 1, .1, 'Acquisition θ'));

As before, this problem can be solved analytically, without simulations.
Unlike before, I'm not going to bother figuring out what it is this time.

Finally, after playing with this interactive slider for a while,
we can identify some effects.

In [ ]:
def do_isolation_curve(mu_incubation_time=1.621, 
                       sigma_incubation_time=0.418, 
                       acquisition_prob=0.9,
                      *args, **kwargs):
    '''Draw isolation time curve for these parameters'''
    isolation_times = infer_isolation_times(mu_incubation_time, sigma_incubation_time, acquisition_prob, n=1000)
    q = np.linspace(0, 1, 50)
    d = np.percentile(isolation_times, q*100)
    d[0] = 0; d[-1] = 20 # Hack to force curve to go to end of plot
    label = 'μ = %.1f, σ = %.1f, θ = %.1f' % (mu_incubation_time, sigma_incubation_time, acquisition_prob)
    plt.plot(d, q, label=label, *args, **kwargs)

In [ ]:
mu_values = [1.4, 1.8]
sigma_values =  [.2, .6]
theta_values = [.5, 1]

mu_default = 1.6
sigma_default = .4
theta_default = .8

default_pars = np.array([mu_default, sigma_default, theta_default])
alt_pars = [mu_values, sigma_values, theta_values]
titles = ['A. Effect of mean incubation time', 
         'B. Effect of variability in incubation time',
         'C. Effect of acquisition probability' ]
pal = iter(sns.palettes.color_palette(n_colors=6))

fig, axes = plt.subplots(1, 3, figsize=(20, 5))
for i in range(3):
    plt.subplot(axes[i])
    for j in range(2):
        pars = np.copy(default_pars)
        pars[i] = alt_pars[i][j]
        do_isolation_curve(*pars, color=next(pal))
    plt.xlabel('Time since first symptoms in house')
    plt.ylabel('P(No longer contagious)')
    plt.xlim(0, 20)
    plt.vlines(14, *plt.ylim(), linestyle='dashed')
    plt.legend()
    plt.title(titles[i])
plt.tight_layout()
plt.show()


- **A.** If the incubation time is longer than we think, people should isolate for longer.
  This should be obvious, since ($\tau_C = \tau_A + \tau_B$)
- **B.** If the incubation time is highly variable, your chances of 
  still being contagious in the first few days are reduced,
  but the chance of still being contagious after 5 days or more is increased.
- **C.** If transmission within a household is slower, the isolation period needs to be longer.
  

## Decision Theory

I was going to finish by talking about *Bayesian decision theory*,
an extension of this framework that allows us to plug these simulations into cost-benefit analysis.
However, this post is already far too long,
so instead I'll close, and maybe return to the topic some other day.

# Endnotes

<h2 id="endnote1">Analytic Solution for <i>Am I Infectious?</i></h2>

We can work out the analytic solution here, if we really want to.
Consider the path diagram below.

<img style="max-height:200px;" src="symptoms.png"/>

If someone has no symptoms by day $d$,
they either are infected but haven't developed symptoms yet (**outcome B**; red),
or they aren't sick (**outcome C**; green).
This means the probability they're infected is


$$
P(\text{Infected}) = \frac{P(B)}{P(B) + P(C)}
$$

Since we believe $\tau$ follows a log normal distribution
$\text{Log-Normal}(1.621, 0.418)$, we know that

$$
p(\tau \gt d) = \int_0^d{f(t\ \mid\ 1.621, 0.418)}dt
$$

where $f(t\ \mid\ \mu, \sigma)$ is the log-normal probability density function.


Putting this together, we find


$$
\begin{align}
P(\text{Infected}) 
 &= \frac{P(B)}{P(B) + P(C)}\\
 &= \frac{\alpha p(\tau > d)}{\alpha p(\tau > d) + 1 - \alpha}\\
 &= \frac{\alpha \int_0^d{f(t\ \mid\ 1.621, 0.418)}dt}
 {\alpha \int_0^d{f(t\ \mid\ 1.621, 0.418)}dt + 1 - \alpha}
\end{align}
$$

Is all this right?
I think so, but this is more algebra than I'm used to doing,
so let's confirm by visualising it again.

In [ ]:
def plot_p_infected_analytic(alpha: float, 
                             incubation_mu: float=1.6, incubation_sigma: float=0.4):
    '''Same as `plot_p_infected`, but using analytic solution.
    '''
    days = np.arange(0, 20)
    incubation_time_distribution = stats.lognorm(loc=0, 
                                                 scale=np.exp(incubation_mu),
                                                 s=incubation_sigma)
    # Find P(𝜏 < d) from the cumulative distribution function, and invert it.
    prob_A = alpha * (1 - incubation_time_distribution.cdf(days))
    prob_B = (1 - alpha)
    prob_infected = prob_A / (prob_A + prob_B)
    plt.plot(days, prob_infected)
    plt.ylim(0, 1)
    plt.xlabel('Days since contact')
    plt.ylabel(u'P(Infected | 😊)')
    plt.show()

interact(plot_p_infected_analytic, 
         alpha = fs(.5, 0, 1, .1, 'α'),
         incubation_mu    = fs(1.6, 0, 5, .1, 'Incubation μ'),
         incubation_sigma = fs(0.4, 0, 1, .1, 'Incubation σ'));



It produces the same results as the simulations, so I'm happy.
Of course, it's possible to mess up the simulations,
and it's possible to mess up the algebra,
but in practice I find I'm much more likely to mess up the algebra.